# CBPP Federal Jobs Guarantee

This analyzes the [CBPP's 2018 Federal Jobs Guarantee proposal](https://www.cbpp.org/research/full-employment/the-federal-job-guarantee-a-policy-to-achieve-permanent-full-employment#_ftn1) from Mark Paul, William Darity, Jr., and Darrick Hamilton. It is primarily described by this table:

![img](https://imgur.com/5Km1yLO.png)

In addition to the \$32,500 average wage, the base wage is \$24,600. The program is available to people age 18 and over.

Modeling the uptake is nontrivial. I estimate each tax unit's benefit as follows:

1. Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.
2. Assign a maximum JG benefit $mb$ equal to $w * n_{18-64}$.
3. Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.
4. Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).
5. Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.
6. Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.
7. Randomly assign each tax unit a participation flag with probability $p$.
8. Add $b$ to the `e00200` of tax units flagged as participating.
9. Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

\* This could be enhanced by splitting `e00200` between `e00200p` and `e00200s` for the taxpayer and spouse, respectively.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import taxcalc_helpers as tch
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
sns.set_style('white')
DPI = 500
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi'] = DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Generate data

### Load data

Generate a set of normal CPS records for 2018 using `Calculator`, then extract the dataframe.

In [152]:
calc = tc.Calculator(records=tc.Records.cps_constructor(), 
                     policy=tc.Policy(), verbose=False)
calc.advance_to_year(2018)

In [153]:
cps_raw_cols = pd.read_csv(
    os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')).columns

In [154]:
df = calc.dataframe(list(cps_raw_cols))

1) Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.

In [155]:
JG_MIN_WAGE = 24600
JG_AVG_WAGE = 32500
jg_max_wage = 39800  # Set to calibrate the average wage to $32,500.
# jg_max_wage = JG_AVG_WAGE + (JG_AVG_WAGE - JG_MIN_WAGE)
print(JG_MIN_WAGE, jg_max_wage)

24600 39800


In [156]:
df['jg_w'] = np.random.randint(low=JG_MIN_WAGE, high=jg_max_wage, 
                               size=df.shape[0])

In [157]:
print(df.jg_w.min(), df.jg_w.max())

24600 39799


2) Assign a maximum JG benefit $mb$ equal to $w * n_{1864}$.

In [158]:
def n65(df):
    return ((df.age_head >= 65).astype(int) + 
            (df.age_spouse >= 65).astype(int) + 
            df.elderly_dependent)

In [159]:
df['n65'] = n65(df)

In [160]:
df['n1864'] = df.n1820 + df.n21 - df.n65
df['jg_mb'] = df.jg_w * df.n1864

3) Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.

In [161]:
df['jg_b'] = np.maximum(0, df.jg_mb - df.e00200)

4) Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $n1864 * b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).

In [162]:
df['jg_fte'] = df.n1864 * df.jg_b / (df.jg_b + df.e00200)

In [163]:
df.jg_fte.mean()

0.48934879818895816

5) Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.

In [164]:
total_potential_jg_fte = (df.jg_fte * df.s006).sum()
total_potential_jg_fte

71767968.53982866

6) Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.

In [165]:
JG_FTE = 9700000
jg_p = JG_FTE / total_potential_jg_fte
jg_p

0.13515778971250728

7) Randomly assign each tax unit a participation flag with probability $p$.

In [166]:
df['jg_participate'] = np.random.rand(df.shape[0]) < jg_p

8) Add $b$ to the `e00200` of tax units flagged as participating, and split the additional amount between `e00200p` and `e00200s` according to the current split.

In [167]:
df['jg'] = df.jg_b * df.jg_participate

In [168]:
(df.jg * df.s006).sum() / 1e9

316.64144925864247

In [169]:
df[df.jg_fte > 0][['e00200', 'jg', 'e00200p', 'jg_fte', 'n1864', 'jg_w', 'jg_mb', 'jg_b']].sort_values('e00200', ascending=False)

,e00200,jg,e00200p,jg_fte,n1864,jg_w,jg_mb,jg_b
98003,211638.82,0.00,0.00,6.34e-02,6.0,35650,213900.0,2261.18
98004,211638.82,0.00,0.00,1.58e-01,6.0,36230,217380.0,5741.18
98006,211638.82,0.00,0.00,1.98e-01,6.0,36475,218850.0,7211.18
81436,169739.80,6910.20,13400.08,1.96e-01,5.0,35330,176650.0,6910.20
81426,169739.80,0.00,13400.08,2.69e-01,5.0,35879,179395.0,9655.20
81424,169739.80,0.00,13400.08,1.30e-01,5.0,34851,174255.0,4515.20
81431,169739.80,0.00,13400.08,2.62e-01,5.0,35827,179135.0,9395.20
81432,169739.80,0.00,13400.08,5.13e-01,5.0,37827,189135.0,19395.20
81428,169739.80,0.00,13400.08,7.21e-02,5.0,34445,172225.0,2485.20
81433,169739.80,0.00,13400.08,1.21e-01,5.0,34790,173950.0,4210.20


In [170]:
(df.jg * df.s006).sum() / (df.jg_participate * df.jg_fte * df.s006).sum()

32539.489542408715

In [171]:
df['e00200_orig'] = df.e00200
df['e00200p_orig'] = df.e00200p
df['e00200s_orig'] = df.e00200s

In [172]:
df['jgp'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200p_orig / df.e00200_orig, 1)
df['jgs'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200s_orig / df.e00200_orig, 0)

In [173]:
df['e00200'] = df.e00200_orig + df.jg
df['e00200p'] = df.e00200p_orig + df.jgp
df['e00200s'] = df.e00200s_orig + df.jgs

In [174]:
df.jgp.sum() / 1e6

730.1471804819707

In [175]:
df.jgs.sum() / 1e6

116.1330323910663

In [176]:
df.jg.sum() / 1e6

846.280212873037

In [177]:
df_noinc = df[(df.e00200_orig == 0)]
(df_noinc.jg * df_noinc.s006).sum() / 1e9

127.37582310368

In [178]:
df_noinc = df[(df.e00200_orig > 0)]
(df_noinc.jg * df_noinc.s006).sum() / 1e9

189.26562615496238

In [203]:
df[['jg', 'jgp', 'jgs', 'e00200', 'e00200p', 'e00200s', 'n1864', 
    'e18400']].head()

,jg,jgp,jgs,e00200,e00200p,e00200s,n1864,e18400
0,0.00,0.00,0.0,48543.44,42211.89,6331.55,2.0,2647.39
1,18580.48,18580.48,0.0,35465.00,35465.00,0.00,1.0,920.03
2,0.00,0.00,0.0,0.00,0.00,0.00,0.0,679.67
3,0.00,0.00,0.0,23216.07,23216.07,0.00,1.0,1265.04
4,0.00,0.00,0.0,0.00,0.00,0.00,0.0,1060.34


People in tax units benefiting from JG.

In [208]:
((df.jg > 0) * df.s006 * df.n1864).sum() / 1e6

15.553943800000003

In [206]:
((df.jg > 0) * df.s006 * df.XTOT).sum() / 1e6

21.82475198

In [180]:
((df.e00200 * df.s006).sum() - (df.e00200_orig * df.s006).sum()) / 1e9

316.6414492586416

9) Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

**TODO**

### Clean data

Fix `e00900` pending https://github.com/open-source-economics/Tax-Calculator/issues/2024.

In [181]:
df['e00900'] = df.e00900p + df.e00900s

## Calculate

Run Tax-Calculator.

In [182]:
df.e00900.sum()

4823675338.633316

In [213]:
recs = tc.Records(data=df, 
                  start_year=2018, 
                  weights=tc.Records.CPS_WEIGHTS_FILENAME,
                  adjust_ratios=tc.Records.CPS_RATIOS_FILENAME,
                  benefits=tc.Records.CPS_BENEFITS_FILENAME)

In [214]:
jg = tch.calc_df(records=recs,
                 year=2018,
                 group_vars=['expanded_income', 'e00200', 'c00100'],
                 metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                              'e00200', 'c07220', 'c11070'])

In [215]:
base = tch.calc_df(records=tc.Records.cps_constructor(), #no_benefits=True),
                   year=2018,
                   group_vars=['expanded_income', 'e00200', 'c00100'],
                   metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                                'e00200', 'c07220', 'c11070'])

Add refundable and nonrefundable CTC components.

In [216]:
jg['ctc_m'] = jg.c07220_m + jg.c11070_m
base['ctc_m'] = base.c07220_m + base.c11070_m

In [217]:
e00200_diff = jg.e00200_m.sum() - base.e00200_m.sum()
((e00200_diff / 1e3).round(1), (e00200_diff / base.e00200_m.sum()).round(3))

(316.6, 0.039)

In [218]:
afti_diff = jg.aftertax_income_m.sum() - base.aftertax_income_m.sum()
(afti_diff / 1e3, afti_diff / base.aftertax_income_m.sum())

(256.1970055245701, 0.022157447359856145)

In [219]:
eitc_diff = jg.eitc_m.sum() - base.eitc_m.sum()
(eitc_diff / 1e3, eitc_diff / base.eitc_m.sum())

(-4.54439052450825, -0.07630014333557093)

In [220]:
ctc_diff = jg.ctc_m.sum() - base.ctc_m.sum()
(ctc_diff / 1e3, ctc_diff / base.ctc_m.sum())

(2.5118629733739217, 0.02915574884641976)

In [221]:
jg.columns

Index(['nu18', 'XTOT', 's006', 'c11070', 'c07220', 'e00200', 'eitc', 'c00100',
       'expanded_income', 'aftertax_income', 's006_m', 'aftertax_income_m',
       'XTOT_m', 'nu18_m', 'eitc_m', 'e00200_m', 'c07220_m', 'c11070_m',
       'ctc_m'],
      dtype='object')

## Analysis

In [222]:
def add_poverty(df):
    EXTREME_POVERTY_LINE = 780
    df['pov_extreme_m'] = df.XTOT_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_child_extreme_m'] = df.nu18_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_10k_m'] = df.XTOT_m * (
        df.aftertax_income < (10000 * df.XTOT))
    df['pov_child_10k_m'] = df.nu18_m * (
        df.aftertax_income < (10000 * df.XTOT))
    df['fpl_m'] = df.XTOT_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpl_child_m'] = df.nu18_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpla_m'] = df.XTOT_m * (df.aftertax_income < tch.fpl(df.XTOT))
    df['fpla_child_m'] = df.nu18_m * (df.aftertax_income < tch.fpl(df.XTOT))

In [223]:
add_poverty(jg)
add_poverty(base)

In [224]:
def print_poverty(numerator, denominator='XTOT_m'):
    print('JG reduced ' + numerator)
    print(jg[numerator].sum() / jg[denominator].sum())
    print(base[numerator].sum() / base[denominator].sum())
    print(jg[numerator].sum() / base[numerator].sum())

In [225]:
print_poverty('fpla_m')

JG reduced fpla_m
0.07673612460967885
0.0882321892438526
0.8697066826438888


In [226]:
print_poverty('fpla_child_m', 'nu18_m')

JG reduced fpla_child_m
0.08547431458620527
0.0960975441798089
0.8894536828774062


In [227]:
print_poverty('fpl_m')

JG reduced fpl_m
0.24233873492393562
0.26931514075715246
0.8998333114232814


In [228]:
print_poverty('fpl_child_m', 'nu18_m')

JG reduced fpl_child_m
0.24992102466007765
0.2821072376781893
0.8859078792766469


In [229]:
print_poverty('pov_extreme_m')

JG reduced pov_extreme_m
0.012076157987397314
0.013908062782076819
0.8682846904429952


In [230]:
print_poverty('pov_child_extreme_m', 'nu18_m')

JG reduced pov_child_extreme_m
0.015021647353044253
0.016866669723265722
0.8906113417471818


In [231]:
print_poverty('pov_10k_m')

JG reduced pov_10k_m
0.1277743039182432
0.14562817122994118
0.8774010058568448


In [232]:
print_poverty('pov_child_10k_m', 'nu18_m')

JG reduced pov_child_10k_m
0.2010102516468281
0.22498692153609093
0.8934308282207566


TODO: How many single-earner 4+ kid tax units?

In [239]:
df[(df.nu18 > 3) & (df.n1864 == 1)].s006.sum()

632007.49

In [238]:
df[(df.nu18 > 3) & (df.n1864 == 1)].s006.sum() / df[(df.n1864 == 1)].s006.sum()

0.007846737463945095

In [245]:
df[(df.nu18 > 3) & (df.n1864 == 1)].nu18_m.sum()

2.7969877800000003

In [243]:
df['nu18_m'] = df.nu18 * df.s006 / 1e6

In [244]:
df[(df.nu18 > 3) & (df.n1864 == 1)].nu18_m.sum() / df[(df.n1864 == 1)].nu18_m.sum()

0.11688084259489248